In [ ]:
# import numpy, astropy and matplotlib for basic functionalities
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
from astropy.constants import c
from astropy.coordinates import Distance
from pathlib import Path
from astropy.table import Table
from agnpy.emission_regions import Blob

# import agnpy classes
import agnpy
from agnpy.spectra import BrokenPowerLaw
from agnpy.synchrotron import Synchrotron
from agnpy.compton import SynchrotronSelfCompton
from agnpy.utils.plot import load_mpl_rc, sed_x_label, sed_y_label, plot_sed

load_mpl_rc()

# import sherpa classes
from sherpa.models import model
from sherpa import data
from sherpa.fit import Fit
from sherpa.stats import Chi2
from sherpa.optmethods import LevMar

In [ ]:
class AgnpySSC(model.RegriddableModel1D):
    """Wrapper of agnpy's synchrotron and SSC classes. 
    A broken power law is assumed for the electron spectrum.
    To limit the span of the parameters space, we fit the log10 of the parameters 
    whose range is expected to cover several orders of magnitudes (normalisation, 
    gammas, size and magnetic field of the blob). 
    """

    def __init__(self, name="ssc"):

        # EED parameters
        self.log10_k_e = model.Parameter(name, "log10_k_e", -2.0, min=-20.0, max=10.0)
        self.p1 = model.Parameter(name, "p1", 2.1, min=-2.0, max=5.0)
        self.p2 = model.Parameter(name, "p2", 3.1, min=-2.0, max=5.0)
        self.log10_gamma_b = model.Parameter(name, "log10_gamma_b", 3, min=1, max=100)
        self.log10_gamma_min = model.Parameter(name, "log10_gamma_min", 1, min=0, max=100)
        self.log10_gamma_max = model.Parameter(name, "log10_gamma_max", 5, min=0, max=100)
        # source general parameters
        self.z = model.Parameter(name, "z", 0.1, min=0.01, max=1)
        self.d_L = model.Parameter(name, "d_L", 1e27, min=1e25, max=1e33, units="cm")
        # emission region parameters
        self.delta_D = model.Parameter(name, "delta_D", 10, min=0, max=100)
        self.log10_B = model.Parameter(name, "log10_B", -2, min=-4, max=2)
        self.t_var = model.Parameter(name, "t_var", 600, min=10, max=np.pi * 1e7, units="s")

        model.RegriddableModel1D.__init__(
            self,
            name,
            (   self.log10_k_e,
                self.p1,
                self.p2,
                self.log10_gamma_b,
                self.log10_gamma_min,
                self.log10_gamma_max,
                self.z,
                self.d_L,
                self.delta_D,
                self.log10_B,
                self.t_var,
            ),
        )

    def calc(self, pars, x):
        """evaluate the model calling the agnpy functions"""
        (   log10_k_e,
            p1,
            p2,
            log10_gamma_b,
            log10_gamma_min,  
            log10_gamma_max,
            z,
            d_L,
            delta_D,
            log10_B,
            t_var,
        ) = pars
        # add units, scale quantities
        x *= u.Hz
        k_e = 10 ** log10_k_e * u.Unit("cm-3")
        gamma_b = 10 ** log10_gamma_b
        gamma_min = 10 ** log10_gamma_min
        gamma_max = 10 ** log10_gamma_max
        B = 10 ** log10_B * u.G
        d_L *= u.cm
        R_b = c.to_value("cm s-1") * t_var * delta_D / (1 + z) * u.cm
        sed_synch = Synchrotron.evaluate_sed_flux(
            x,
            z,
            d_L,
            delta_D,
            B,
            R_b,
            BrokenPowerLaw,
            k_e,
            p1,
            p2,
            gamma_b,
            gamma_min,
            gamma_max,
        )
        #sed_ssc = SynchrotronSelfCompton.evaluate_sed_flux(
        #    x,
        #    z,
        #    d_L,
        #    delta_D,
        #    B,
        #    R_b,
        #    BrokenPowerLaw,
        #    k_e,
        #    p1,
        #    p2,
        #    gamma_b,
        #    gamma_min,
        #    gamma_max,
        #)
        return sed_synch   #  return sed_synch + sed_ssc

In [ ]:
#######################################################################################################
### read data
#######################################################################################################
# Mrk 421
#source = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/Mrk421_example.csv') 

# My source
source = pd.read_csv('/Users/87steven/Downloads/J1104+3812_flux.csv') 

freq = source['freq'].values
flux = source['flux'].values
flux_err = source['flux_err'].values

### set flux error which is nan to 0.01
fluxerr_nan = np.where( np.isnan(flux_err ))[0]
flux_err[fluxerr_nan] = 0.01
### find flux index which is not nan
flux_NOTnan = np.where( (~np.isnan(flux)) & (flux > 0) )[0]
### save new data into array
freq = freq[flux_NOTnan]
flux = flux[flux_NOTnan]
flux_err = flux_err[flux_NOTnan]

#######################################################################################################
### systematic errors
#######################################################################################################
# array of systematic errors, will just be summed in quadrature to the statistical error
# we assume
# - 30% on VHE gamma-ray instruments
# - 10% on HE gamma-ray instruments
# - 10% on X-ray instruments
# - 5% on lower-energy instruments

### define energy ranges
nu_vhe = 2.42E25 # [Hz]
nu_he = 2.42E22 # [Hz]
nu_x_ray_max = 4.25E19 # [Hz]
nu_x_ray_min = 7.25E16 # [Hz]
vhe_gamma = freq >= nu_vhe
he_gamma = (freq >= nu_he) * (freq  < nu_vhe)
x_ray = (freq  >= nu_x_ray_min) * (freq  < nu_x_ray_max)
uv_to_radio = freq  < nu_x_ray_min

### declare systematics
y_err_syst = np.zeros(len(freq ))
y_err_syst[vhe_gamma] = 0.30
y_err_syst[he_gamma] = 0.10
y_err_syst[x_ray] = 0.10
y_err_syst[uv_to_radio] = 0.05
y_err_syst = flux * y_err_syst

### Constrain flux range
index = np.where(freq <= 1.0E19)[0]

freq = freq[index]
flux = flux[index]
flux_err = flux_err[index]
y_err_syst = y_err_syst[index]

# define the data1D object containing it
sed = data.Data1D("sed", freq, flux, staterror = flux_err, syserror = y_err_syst)  # , syserror = y_err_syst

In [ ]:
#######################################################################################################
# test parameters
#######################################################################################################
break_val = np.linspace(1.0E3, 1.0E5, 100)
min_val = np.linspace(10, 1.0E4, 100)
max_val = np.linspace(1.0E6, 1.0E8, 100)
B_val = np.linspace(0.01, 1, 100)
delta_D_val = np.linspace(1, 100, 100)
t_var_val = np.linspace(1, 24, 24)
k_e_val = np.linspace(1.0E-5, 1.0E-9, 5)

def ClearData():
    data = {
    'z': [],
    'k_e': [],
    'p1': [],
    'p2': [],
    'gamma_b': [],
    'gamma_min': [],
    'gamma_max': [],
    'B': [],
    'delta_D': [],
    't_var': [],
    'R_b': [],
    'chi_squ': []
      }
    return data

df = ClearData()


In [ ]:
z_array = []
k_e_array = []
b_array = []
min_array = []
max_array = []
p1_array = []
p2_array = []
B_array = []
delta_D_array = []
R_b_array = []
chi_square_array = []
t_var_array = []

for i in range(14, 28): # 14-28
    #######################################################################################################
    ### find frequency of maximum flux between 1.0E12 ~ 1.0E20
    #######################################################################################################
    index = np.where((freq > 1.0E12) & (freq <= 1.0E20))[0]

    max_flux_index = np.where(flux == max(flux[index]))[0]

    min_freq = freq[0]
    max_freq = freq[max_flux_index][0]

    #######################################################################################################
    # declare parameters
    #######################################################################################################
    agnpy_ssc = AgnpySSC()
    # initialise parameters (parameters from Table 4 and Figure 11 of Abdo 2011)
    z = 0.031                            # redshift (0.0308)
    d_L = Distance(z=z).to("cm")          # luminosity distance [cm]
    p1 = 2.8                             # spectral index (2.02)
    p2 = 2.8                            # spectral index (3.43)
    
    ### magnetic field strength
    log10_B = np.log10(B_val[i])
    
    gamma_b_min = np.sqrt(min_freq/(4.2E6*B_val[i]))
    gamma_b_val = np.sqrt(max_freq/(4.2E6*B_val[i]))

    log10_gamma_b = np.log10(gamma_b_val)       # lorentz break factor (5 = np.log10(1.0E5))
    log10_gamma_min = np.log10(gamma_b_min)       # minimum lorentz factor (500)
    log10_gamma_max = np.log10(gamma_b_val*100)     # maximum lorentz factor (1.0E6)
    
    for m in range(0, 5):
        
        log10_k_e = np.log10(k_e_val[m])         #  (-7.9 = np.log10(1.26E-8))
    
        for j in range(0, 24):
        
            t_var = 60*60*t_var_val[j]                   # variability timescale [sec] (86400)

            for k in range(0, 100):
            
                #delta_D = 18
                delta_D = delta_D_val[k]                          # doppler factor (free parameter) (18)
            
                R_b = c*t_var*delta_D/(1+z)*100       # radius [cm] ====> Doppler factor is include in here (5.2E16 )
            
                print("i = ", i, ", m = ", m, ", j = ", j, ", k = ", k)
        
                #######################################################################################################
                ### enter parameters into model
                #######################################################################################################
                # - AGN parameters
                agnpy_ssc.z = z                 # redshift 
                agnpy_ssc.z.freeze()
                agnpy_ssc.d_L = d_L             # luminosity distance [cgs]
                agnpy_ssc.d_L.freeze()
                # - blob parameters
                agnpy_ssc.delta_D = delta_D     # doppler factor (free parameter)
                agnpy_ssc.log10_B = log10_B     # magnetic field sterength [G] (free parameter)
                agnpy_ssc.t_var = t_var         # variability timescale [sec]
                agnpy_ssc.t_var.freeze()
                # - EED
                agnpy_ssc.log10_k_e = log10_k_e # 
                agnpy_ssc.p1 = p1               # spectral index
                agnpy_ssc.p2 = p2               # spectral index
                agnpy_ssc.log10_gamma_b = log10_gamma_b       # lorentz break factor
                agnpy_ssc.log10_gamma_min = log10_gamma_min   # minimum lorentz factor
                agnpy_ssc.log10_gamma_min.freeze()
                agnpy_ssc.log10_gamma_max = log10_gamma_max   # maximum lorentz factor
                agnpy_ssc.log10_gamma_max.freeze()

                #######################################################################################################
                ### fit using the Levenberg-Marquardt optimiser
                #######################################################################################################
                fitter = Fit(sed, agnpy_ssc, stat = Chi2(), method = LevMar())
                # Set minimum and maximum frequency to proced model fit
                min_x = 1.0E8
                max_x = 1.0E20
                sed.notice(min_x, max_x)

                #######################################################################################################
                ### perform model fit
                #######################################################################################################
                result = fitter.fit()
                print("Model fit succesful??", result.succeeded)
            
                z_array.append(z)
                k_e_array.append(k_e_val)
                p1_array.append(p1)
                p2_array.append(p2)
                b_array.append(gamma_b_val)
                min_array.append(gamma_b_min)
                max_array.append(gamma_b_val*100)
                B_array.append(B_val[i])
                t_var_array.append(t_var_val[j])
                delta_D_array.append(delta_D_val[k])
                R_b_array.append(R_b)
                chi_square_array.append(float(result.rstat))
            
                print('=================================================')
            
for i in range(0, len(k_e_array)):
    df['z'].append(z_array[i]) 
    df['k_e'].append(k_e_array[i]) 
    df['p1'].append(p1_array[i])
    df['p2'].append(p2_array[i])
    df['gamma_b'].append(b_array[i]) 
    df['gamma_min'].append(min_array[i])
    df['gamma_max'].append(max_array[i])
    df['B'].append(B_array[i]) 
    df['t_var'].append(t_var_array[i])
    df['delta_D'].append(delta_D_array[i])
    df['R_b'].append(R_b_array[i])
    df['chi_squ'].append(chi_square_array[i])

#CSVfile = f'/Users/hsu/Documents/Blazar SED code and data/J1104+3812_SED_parameters2.csv'   
#dff = pd.DataFrame(df)      
#dff.to_csv(CSVfile, index = False)            
            